<a href="https://colab.research.google.com/github/YuukaObara/gbm/blob/main/%E6%B0%91%E6%B3%8A%E3%82%B5%E3%83%BC%E3%83%93%E3%82%B9%E3%81%AE%E5%AE%BF%E6%B3%8A%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%AC%EF%BC%88Python%EF%BC%89_%E5%B0%8F%E5%8E%9F%E5%84%AA%E4%BD%B3_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **データの読み込み**

In [ ]:
#ドライブと接続する
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
!pip install ydata-profiling
!pip install sweetviz
import ydata_profiling
!pip install numpy==1.24.3
import numpy as np
import pandas as pd
#Label Encoding
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import phik
import sweetviz as sv
from sklearn.metrics import r2_score
#Count Encoding
!pip install category_encoders
from category_encoders import OrdinalEncoder, CountEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
#optuna
!pip install optuna
import optuna
import lightgbm as lgb

In [ ]:
#【練習問題】民泊サービスの宿泊価格予測(https://signate.jp/competitions/266)のデータ使用
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/民泊サービスの宿泊価格予測/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/民泊サービスの宿泊価格予測/test.csv')

## **データの可視化**

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
# 施設数：都市別・施設規模別
# accommodatesを1, 2, 3以上に分けるための新しいカラムを作成
df_train['accommodates_group'] = pd.cut(df_train['accommodates'], bins=[0, 1, 2, float('inf')], labels=['1', '2', '3+'])

# cityと新しいカラムでグループ化し、数をカウント
accommodates_city_counts = df_train.groupby(['city', 'accommodates_group'])['accommodates_group'].count().unstack(fill_value=0)

# 結果を表示
accommodates_city_counts

In [ ]:
# 施設数（％表示）：都市別・施設規模別
# 各cityの合計値を計算
city_totals = accommodates_city_counts.sum(axis=1)

# 各グループの値をcityの合計値で割ってパーセンテージを計算
accommodates_city_percentage = accommodates_city_counts.div(city_totals, axis=0) * 100

# 結果を表示
accommodates_city_percentage.round(1)

In [ ]:
# 施設数（％表示）円グラフ：都市別・施設規模別
for city in accommodates_city_percentage.index:
    plt.figure(figsize=(3, 3))  # グラフのサイズを設定
    plt.pie(accommodates_city_percentage.loc[city],
            labels=accommodates_city_percentage.columns,
            autopct='%1.1f%%',  # パーセンテージの表示形式
            startangle=90)  # グラフの開始角度
    plt.title(f'{city}')  # グラフのタイトル
    plt.show()

In [ ]:
# 平均宿泊価格：都市別・施設規模別
# cityとaccommodates_groupでグループ化し、yの平均を計算
city_accommodates_y_mean = df_train.groupby(['city', 'accommodates_group'])['y'].mean().unstack()

# 全都市の平均を計算
all_city_mean = df_train.groupby('accommodates_group')['y'].mean()

# 全都市の平均を新しい行として追加
city_accommodates_y_mean.loc['All Cities'] = all_city_mean

# 結果を表示
city_accommodates_y_mean.round(1)

In [ ]:
# 宿泊価格中央値：都市別・施設規模別
# cityとaccommodates_groupでグループ化し、yの中央値を計算
city_accommodates_y_median = df_train.groupby(['city', 'accommodates_group'])['y'].median().unstack()

# 全都市の平均を計算
all_city_median = df_train.groupby('accommodates_group')['y'].median()

# 全都市の平均を新しい行として追加
city_accommodates_y_median.loc['All Cities'] = all_city_median

# 結果を表示
city_accommodates_y_median.round(1)

In [ ]:
# 施設数：都市別
df_train['city'].value_counts()

In [ ]:
# 宿泊価格の平均・中央値：都市別
# cityの値ごとのyの平均と中央値を計算
city_stats = df_train.groupby('city')['y'].agg(['mean', 'median'])
# 中央値で降順にソート
city_stats = city_stats.sort_values('median', ascending=False)

# 折れ線グラフで表示
plt.figure(figsize=(8, 5))  # グラフのサイズを設定
plt.plot(city_stats.index, city_stats['mean'], marker='o', label='mean')
plt.plot(city_stats.index, city_stats['median'], marker='x', label='median')
plt.xlabel('city')  # x軸ラベル
plt.ylabel('y')  # y軸ラベル
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#相関係数ヒートマップ　数値型のみ
sns.heatmap(data=df_train.select_dtypes(include=['number']).corr().round(2), annot=True, vmax=1, vmin=-1, cmap='bwr')

## **データの変換**

In [ ]:
# 各都市名のカラムを作成
for city in ['NYC', 'LA', 'SF', 'DC', 'Chicago', 'Boston']:
    df_train[city] = df_train['city'].str.contains(city)
for city in ['NYC', 'LA', 'SF', 'DC', 'Chicago', 'Boston']:
    df_test[city] = df_test['city'].str.contains(city)

In [ ]:
# amenities列内の単語出現回数TOP5を出力
from collections import Counter
# すべてのamenitiesの値を結合
all_amenities = ' '.join(df_train['amenities'].astype(str).tolist())
# 単語に分割
words = all_amenities.split(',')
# 出現回数をカウント
word_counts = Counter(words)
# 出現頻度の高い順に単語と出現回数を表示
for word, count in word_counts.most_common(5):
    print(f'{word}: {count}')

In [ ]:
# 各アメニティ名のカラムを作成
for amenity in ['Air conditioning', 'Internet', 'TV', 'Free parking on premises', 'Heating', 'Smoke detector', 'Essentials']:
    df_train[amenity] = df_train['amenities'].str.contains(amenity)
for amenity in ['Air conditioning', 'Internet', 'TV', 'Free parking on premises', 'Heating', 'Smoke detector', 'Essentials']:
    df_test[amenity] = df_test['amenities'].str.contains(amenity)

In [ ]:
# 今回の分析で使わないカラムの削除、数値の変換、エンコーディング、カテゴリ変数をカテゴリ型に変換
def data_pre(df): #データの前処理用の関数を作成
    #列を削除する id, name, description, neighbourhood(93%が欠損値), host_has_profile_pic(99%以上が同じ値)
    df = df.drop(columns = ['id', 'name', 'description', 'thumbnail_url', 'host_has_profile_pic', 'cleaning_fee', 'host_identity_verified', 'cancellation_policy',
                            'instant_bookable', 'bed_type', 'zipcode', 'number_of_reviews', 'room_type', 'neighbourhood', 'property_type', 'city', 'beds', 'amenities', 'first_review'])

    #host_response_rateをパーセンテージから割合にする
    df['host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float)/100
    #'host_since' を日付型に変換　最新の日付から経過日数を計算し代入
    df['host_since'] = pd.to_datetime(df['host_since'])
    newest_date = df['host_since'].max()
    df['host_since'] = (newest_date - df['host_since']).dt.days
    # 'last_review' を日付型に変換　最新の日付から経過日数を計算し代入
    df['last_review'] = pd.to_datetime(df['last_review'])
    newest_date = df['last_review'].max()
    df['last_review'] = (newest_date - df['last_review']).dt.days
    # # 'first_review' を日付型に変換　最新の日付から経過日数を計算し代入
    # df['first_review'] = pd.to_datetime(df['first_review'])
    # newest_date = df['first_review'].max()
    # df['first_review'] = (newest_date - df['first_review']).dt.days

    #Encoding　カテゴリ型に変換
    def encode_categorical(df, cols):
        for col in cols:
            #Label Encoding
            le = LabelEncoder()
            not_null = df[col][df[col].notnull()]
            df[col] = pd.Series(le.fit_transform(not_null),index = not_null.index)
            df[col] = df[col].astype("category")
        return df
    df = encode_categorical(df, cols = ['NYC', 'LA', 'SF', 'DC', 'Chicago', 'Boston', 'Air conditioning', 'Internet', 'TV', 'Free parking on premises', 'Heating', 'Smoke detector', 'Essentials'])
    return df

In [ ]:
# 変換実行
df_train_encoded = data_pre(df_train)
df_test_encoded = data_pre(df_test)

In [ ]:
#Count Encoding
#cols = ['Air conditioning', 'Internet', 'TV', 'Free parking on premises', 'Heating', 'Smoke detector', 'Essentials']
# 各カラムにCount Encodingを適用
#def count_encoding(df, cols):
#    for col in cols:
#        encoder = CountEncoder()  # CountEncoderのインスタンスを作成
#        encoder.fit(df[col])  # 学習データでencoderを学習
#        # 学習データとテストデータに変換を適用
#        df[f'{col}_count'] = encoder.transform(df[col])
#        # 元のカラムを削除
#        df = df.drop(columns=[col])
#    return df
#df_train_encoded = count_encoding(df_train_encoded, cols)
#df_test_encoded = count_encoding(df_test_encoded, cols)

In [ ]:
#One hot Encoding
#from sklearn.preprocessing import OneHotEncoder
#encoded_data = pd.get_dummies(df_train_encoded['room_type'])
#df_train_encoded = pd.concat([df_train_encoded, encoded_data], axis=1)
#df_train_encoded = df_train_encoded.drop(columns=['room_type'])
#encoded_data = pd.get_dummies(df_test_encoded['room_type'])
#df_test_encoded = pd.concat([df_test_encoded, encoded_data], axis=1)
#df_test_encoded = df_test_encoded.drop(columns=['room_type'])

In [ ]:
df_train_encoded

## **ツールによる可視化**

In [ ]:
report = sv.compare(df_train_encoded, df_test_encoded, target_feat="y")

In [ ]:
report.show_notebook()

In [ ]:
df_train_encoded.profile_report()

## **モデル構築**

In [ ]:
# # LightGBMのモデル構築、チューニング、評価

# from sklearn.model_selection import train_test_split, cross_val_score

# #訓練データを説明変数と目的変数に分割
# col = "y"
# y = df_train_encoded[col] #yのみを目的変数にする
# X = df_train_encoded.drop(col, axis=1) #col以外のカラムを説明変数にする

# # データを訓練用とテスト用に分割
# # 訓練データでモデルを学習させ、テストデータで性能を確認する
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# def objective(trial):
#     # Optunaでチューニングするパラメータを定義
#     # trial.suggest_XXXを使って、探索するパラメータの範囲や値を指定
#     params = {
#         'objective': 'regression',  # 回帰問題を指定
#         'metric': 'rmse',  # モデルの性能指標としてRMSE（平均二乗平方根誤差）を使用
#         # 'n_estimators': trial.suggest_int('n_estimators', 200, 500),  # 決定木の数（データサイズに合わせ調整）
#         'max_depth': trial.suggest_int('max_depth', 3, 5),  # 決定木の深さ（浅めに設定）
#         # 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),  # 学習率（データサイズに基づき調整）
#         'num_leaves': trial.suggest_int('num_leaves', 10, 50),  # 決定木のリーフ数（データ特徴量数に適合）
#         # 'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),  # 特徴量のサブサンプル比率
#         # 'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),  # データサンプルのサブサンプル比率
#         # 'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),  # バギングの頻度（頻度を高めに設定）
#         # 'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 1.0, log=True),  # L1正則化の強さ
#         # 'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 1.0, log=True),  # L2正則化の強さ
#         # 'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 1.0, log=True),  # 子ノードで必要な最小データ量
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 30)
#     }

#     # LightGBMのモデルを初期化
#     model = lgb.LGBMRegressor(**params, random_state=42)

#     # クロスバリデーションを使用してモデルの性能を評価
#     # 訓練データをさらに分割し、複数回学習・評価を繰り返して平均スコアを算出
#     score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

#     # クロスバリデーションのスコア（負のMSE）の平均値を返す
#     # Optunaはこの値を元にパラメータを最適化する
#     return np.mean(score)

# # Optunaのチューニングを実行
# # maximize: 負のMSEの平均値を最大化する（つまり、MSEを最小化することに相当）
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)  # 50回の試行で最適化を行う

# # 最適なパラメータを取得
# best_params = study.best_params
# print("Best Parameters:", best_params)

# # 最適なパラメータでLightGBMモデルを学習
# best_model = lgb.LGBMRegressor(**best_params, random_state=42)
# best_model.fit(X_train, y_train)  # 訓練データでモデルを学習

# # テストデータでの評価
# predictions = best_model.predict(X_test)  # 学習したモデルでテストデータを予測
# rmse = mean_squared_error(y_test, predictions)  # RMSEを計算
# print("Test RMSE:", np.sqrt(rmse))  # テストデータでの性能を表示


In [ ]:
# print(best_params)
# print("Test RMSE:", np.sqrt(rmse))

In [ ]:
# チューニングによるベストパラメータを用いて新しいコードに適用、評価

# 訓練データを説明変数と目的変数に分割
col = "y"
y = df_train_encoded[col]  # yのみを目的変数にする
X = df_train_encoded.drop(col, axis=1)  # col以外のカラムを説明変数にする

# 分割数、学習回数、実行メッセージを出さない
FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1

# ベストパラメータを適用
# params = best_params # ここでベストパラメータを使用しています
params = {}
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['verbose'] = -1
params['max_depth'] = 5
params['num_leaves'] = 27
params['min_data_in_leaf'] = 22

valid_scores_rmse = []
valid_scores_r2 = []
models = []
evals_result = {}
evals_result_for_learning_curve = []
kf = KFold(n_splits=FOLD, shuffle=True, random_state=4)

# データの分割と学習
for fold, (train_indices, valid_indices) in enumerate(kf.split(X)):
    X_train, X_valid = X.iloc[train_indices], X.iloc[valid_indices]
    y_train, y_valid = y.iloc[train_indices], y.iloc[valid_indices]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        num_boost_round=NUM_ROUND,
        callbacks=[lgb.log_evaluation(VERBOSE_EVAL), lgb.record_evaluation(evals_result), lgb.early_stopping(stopping_rounds=100)],
    )

    y_valid_pred = model.predict(X_valid)

    #学習曲線
    evals_result_for_learning_curve.append(evals_result.copy())
    #RMSE
    score_rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
    print(f'fold {fold} RMSE: {score_rmse}')
    valid_scores_rmse.append(score_rmse)
    #adjustedR2
    n = len(y)
    k = X.shape[1]
    score_r2 = (1 - (1 - r2_score(y_valid, y_valid_pred)) * (n - 1) / (n - k - 1))
    print(f'fold {fold} adjustedR2: {score_r2}')
    valid_scores_r2.append(score_r2)

    models.append(model)

#RMSE出力
cv_score = np.mean(valid_scores_rmse)
print(f'CV score: {cv_score}')
#adjustedR2出力
score_r2 = np.mean(valid_scores_r2)
print(f'adjustedR2: {score_r2}')

## **予測値算出**

In [ ]:
# 予測値算出 全モデルの平均
predict = np.mean([model.predict(df_test_encoded) for model in models], axis=0)
predict

In [ ]:
#予測値を入れるカラムを作る
df_test["y"] = predict
#予測値のカラムのみをcsvとして出力
df_test["y"].to_csv("submit_test.csv", header = False)
from google.colab import files
files.download("submit_test.csv")

In [ ]:
#予測値表示
df_test

## **評価指標可視化**

In [ ]:
# 学習曲線
lgb.plot_metric(evals_result, metric='rmse')

In [ ]:
# SHAP値
!pip install shap

import shap

# notebook内でJavascriptを動かすために入れる
shap.initjs()

# TreeExplainerは、決定木系(XGBoost、lightBGM等含む)のモデルのSHAP値を取得するもの。
explainer = shap.TreeExplainer(model=models[-1])
explainer.expected_value

In [ ]:
# SHAP値の計算
X_valid_shap = X_valid.copy().reset_index(drop=True)
shap_values = explainer.shap_values(X=X_valid_shap)

print(X_valid_shap.shape)
print(shap_values.shape)
print(shap_values[0])#テストデータの0番目の要素を出力

In [ ]:
# 欠損値の数
df_train_encoded.isnull().sum()

In [ ]:
# SHAP サマリープロット
shap.summary_plot(shap_values, X_valid_shap)

In [ ]:
# SHAP サマリープロット（絶対値）
shap.summary_plot(shap_values, X_valid_shap, plot_type='bar')